In [14]:
dic_when_fincausal = {
'2010': ['2010'],
'2010, 2011, 2012': ['2010', '2011', '2012'],
'2010, 2011, and 2012': ['2010', '2011', '2012'],
'2010-2011': ['2010', '2011'],
'2011': ['2011'],
'2011 (and fourth quarter of 2010)' : ['2010', '2011'],
'2011 and 2010': ['2010', '2011'],
'2011 and 2012': ['2011', '2012'],
'2011, 2010': ['2010', '2011'],
'2011, 2012' : ['2011', '2012'],
'2011-2012' : ['2011', '2012'],
'2012': ['2012'],
'2012 and 2011' : ['2011', '2012'],
'2012, 2011': ['2011', '2012'],
'2012-12-31': ['2012'],
'2013' : ['2013'],
'2013, 2012' : ['2012', '2013'],
'2013-12-31': ['2013'],
'2018': ['2018'],
'2018, 2019, 2020': ['2018', '2019', '2020'],
'2018-01-01' : ['2018'],
'2019' : ['2019'],
'2019 and 2018' : ['2018', '2019'],
'2019 compared to 2018': ['2018', '2019'],
'2019, 2018' : ['2018', '2019'],
'2019-04-18' : ['2019'],
'2019-06-23' : ['2019'],
'2019-12-31' : ['2019'],
'2019-2020' : ['2019', '2020'],
'2020': ['2020'],
'2020 (compared to 2019)': ['2019', '2020'],
'2020 and 2019': ['2019', '2020'],
'2020, 2019': ['2019', '2020'],
'2020, 2019, and 2018': ['2018','2019', '2020'],
'2020-12-27': ['2020'],
'2020-12-31': ['2020'],
'2020-Q4': ['2020'],
'2021' : ['2021'],
'2021 and the years ahead': ['2021'],
'As of December 31, 2012 and 2011' : ['2011', '2012'],
'As of December 31, 2020' : ['2020'],
'August 2011 - December 31, 2012' : ['2011','2012'],
'Beginning in March' : ['Beginning in March'],
'Beginning in March 2020' : ['2020'],
'By the end of 2021' : ['2021'],
'December 2012' : ['2012'],
'December 31, 2011' : ['2011'],
'December 31, 2012' : ['2012'],
'December 31, 2012 and 2011' : ['2011', '2012'],
'December 31, 2013' : ['2013'],
'December 31, 2019' : ['2019'],
'December 31, 2020' : ['2020'],
'During 2011' : ['2011'],
'During 2012' : ['2012'],
'During 2020' : ['2020'],
'During the winter and spring' : ['During the winter and spring'],
'Current year' : ['Current year'],
'Current year and prior year' : ['Current year and prior year'],
'December 1' : ['December 1'],
'During the year ended December 31, 2020' : ['2020'],
'End of 2013' : ['2013'],
'End of 2019' : ['2019'],
'February 2020' : ['2020'],
'First and second quarters of the year' : ['First and second quarters of the year'],
'First half of 2013' : ['2013'],
'First half of 2020' : ['2020'],
'First half of the year' : ['First half of the year'],
'First quarter of 2012' : ['2012'],
'For the year ended December 31, 2020' : ['2020'],
'Fourth quarter of 2020' : ['2020'],
'January 2012' : ['2012'],
'January 2013' : ['2013'],
'January 2021' : ['2021'],
'July 2012, during 2012' : ['2012'],
'June 2012' : ['2012'],
'June 2019' : ['2019'],
'June 2019, February 2020' : ['2019', '2020'],
'June 2020' : ['2020'],
'March 2020, April 2020' : ['2020'],
'Primarily during the first half of the year' : ['Primarily during the first half of the year'],
'Prior to June 2019' : ['2019'],
'Second half of 2011 and first quarter of 2012' : ['2011', '2012'],
'Third quarter of 2013' : ['2013'],
'Third quarter of 2020' : ['2020'],
'Through December 31, 2012' : ['2012'],
'Through the end of 2020' : ['2020'] ,
'Year ended December 31, 2019' : ['2019'],
'Years ended December 31, 2020 and 2019' : ['2019', '2020'],
'Years ended December 31, 2020, 2019 and 2018' : ['2018', '2019', '2020']}

dic_who_fincausal = {
'Airlines' : 'Airlines',
'American Airlines' : 'Airlines',
'Architectural Coatings – EMEA' : 'Architectural Coatings – EMEA',
'Automotive Group' : 'Automotive',
'Automotive OEMs' : 'Automotive',
'Automotive group' : 'Automotive',
'Automotive segment' : 'Automotive',
'Banking industry' : 'Banking industry',
'Borrowers' : 'Borrowers',
'Businesses' : 'Businesses',
'Businesses and consumers' : 'Businesses',
'Businesses in a specific segment' : 'Businesses',
'CAS Medical Systems, Inc. (CASMED)' : 'CAS Medical Systems, Inc. (CASMED)',
'Calphalon' : 'Calphalon',
'Casinos' : 'Casinos',
'Clinical Labs' : 'Clinical Labs',
'Coleman' : 'Coleman',
'Companies' : 'Company',
'Company' : 'Company',
'Company (not specifically named)' : 'Company',
'Company (not specified)' : 'Company',
'Company employees' : 'Company',
'Company selling enhanced surgical recovery products' : 'Company',
'Consumer Banking and Wealth' : 'Consumers',
'Consumers' : 'Consumers',
'Corporate and Commercial Banking' : 'Corporations',
'Corporate and Institutional Group' : 'Corporations',
'Corporations' : 'Corporations',
'Currencies' : 'Currencies',
'Customer' : 'Customers',
'Customers' : 'Customers',
'Delta Air Lines' : 'Airlines',
'Delta Air Lines and the U.S. Department of the Treasury' : 'Airlines',
'Dx' : 'Dx',
'Employers' : 'Employers',
'Encore Boston Harbor' : 'Encore Boston Harbor',
'Encore Boston Harbor developers' : 'Encore Boston Harbor',
'Every coatings business except aerospace' : 'Every coatings business except aerospace',
'Federal Reserve' : 'Federal Reserve',
'Financial institutions' : 'Financial institutions',
'General Electric' : 'General Electric',
'Glass segment' : 'Glass segment',
'Global population' : 'Global population',
'Grupo Aeroméxico, LATAM' :  'Airlines',
'Home Fragrance business unit' : 'Home Fragrance business unit',
'Home Solutions segment' : 'Home Solutions segment',
'Homeowners' : 'Homeowners',
'Hunter Douglas' : 'Hunter Douglas',
'Industrial Coatings segment' : 'Industrial segment',
'Industrial Parts Group' : 'Industrial segment',
'Industrial segment' : 'Industrial segment',
'Insurance companies' : 'Insurance companies',
'International Air Transport Association (IATA)' : 'Airlines',
'Investors' : 'Investors',
'Investors holding securities portfolios' : 'Investors',
'LHFS' : 'LHFS',
'Labcorp' : 'Labcorp',
'Las Vegas Operations and Macau Operations' : 'Operations',
'Macau Operations, Las Vegas Operations, Encore Boston Harbor' : 'Operations',
'Many countries' : 'Many countries',
'Massachusetts Gaming Commission' : 'Massachusetts Gaming Commission',
'Medtronic' : 'Medtronic',
'North American and European markets' : 'North American and European markets',
'North American and international businesses' : 'North American and international businesses',
'Optical and Specialty Materials' : 'Optical and Specialty Materials',
'PPG' : 'PPG',
'Paper Mate' : 'Paper Mate',
'Project Renewal' : 'Project Renewal',
'Project Renewal and the European Transformation Plan' : 'Project Renewal and the European Transformation Plan',
'Refineries' : 'Refineries',
'Refinery' : 'Refineries',
'Restaurants, Las Vegas Operations' : 'Restaurants, Las Vegas Operations',
'Retailers in North America and Asia Pacific markets' : 'Retailers in North America and Asia Pacific markets',
"S&P, Moody's" : "S&P, Moody's",
'SG&A (Selling, General & Administrative Expenses)' : 'SG&A (Selling, General & Administrative Expenses)',
'Shareholders' : 'Shareholders',
'Silicas business' : 'Silicas business',
"Standard & Poor's, Fitch" : "Standard & Poor's, Fitch",
'SunTrust' : 'SunTrust',
'Technology businesses' : 'Technology businesses',
'The Company' : 'The Company',
"The Company's Chief Executive Officer" : 'The Company',
'The Company, Dx' : 'The Company',
'The Company, Industrial Group' : 'The Company',
'The Company, holders of Convertible Notes' : 'The Company',
'The International Monetary Fund (IMF)' : 'The International Monetary Fund (IMF)',
'The World Bank' : 'The World Bank',
'The company (speaker not identified)' : 'The Company',
'The company (speaker)' : 'The Company',
'The company receiving payroll support payments' : 'The Company',
'The company reporting earnings' : 'The Company',
'The company reporting the information' : 'The Company',
'The refinery' : 'The refinery',
'The speaker (likely a company representative)' : 'The speaker',
"The speaker's company (not explicitly named)" : 'The speaker',
'Third-party refineries' : 'The refinery',
'Transitions' : 'Transitions',
'Truist' : 'Truist',
'U.S. Department of the Treasury' : 'U.S. Government',
'U.S. dollar' : 'U.S. Government',
'US Congress' : 'U.S. Government',
'US corporations' : 'U.S. Government',
'US lawmakers' : 'U.S. Government',
'United States Trade Representative (USTR)' : 'U.S. Government',
'Wynn Las Vegas' : 'Wynn',
'Wynn Macau' : 'Wynn',
"Wynn Palace, Wynn Macau, the Company's Las Vegas Operations, and Encore Boston Harbor" : 'Wynn',
'Wynn Resorts' : 'Wynn',
'Wynn Resorts, Limited' : 'Wynn',
'Yankee Candle' : 'Yankee Candle',
'Zambia and its creditors' : 'Zambia and its creditors'}

dic_where_fincausal = {
'Asia' : 'Asia',
'Asia Pacific' : 'Asia',
'Asian marine new-build market' : 'Asia',
'Boston Harbor' : 'Boston Harbor',
'Countries in which PPG operates' : 'Countries in which PPG operates',
'Domestic region' : 'Domestic region',
'EMEA (Europe, Middle East, and Africa)' : 'EMEA (Europe, Middle East, and Africa)',
'EMEA segment' : 'EMEA (Europe, Middle East, and Africa)',
'Emerging markets' : 'Emerging markets',
'Encore Boston Harbor, Las Vegas Operations' : 'Encore Boston Harbor, Las Vegas Operations',
'Europe' : 'Europe',
'Europe and the United States' : 'Europe and the United States',
'Europe, Middle East, and Africa region' : 'EMEA (Europe, Middle East, and Africa)',
'European markets' : 'Europe',
'European region and Brazil' : 'European region and Brazil',
'Former chromium manufacturing plant and associated sites in New Jersey' : 'Former chromium manufacturing plant and associated sites in New Jersey',
'Global' : 'Global',
'Jersey City, New Jersey' : 'Jersey City, New Jersey',
'Las Vegas' : 'Las Vegas',
'Las Vegas Operations' : 'Las Vegas',
'Las Vegas Operations and Encore Boston Harbor' : 'Las Vegas',
'Las Vegas Operations, Wynn Palace, Wynn Macau, and Encore Boston Harbor' : 'Las Vegas',
'Las Vegas and Macau' : 'Las Vegas',
'Las Vegas, Macau' : 'Las Vegas',
'Las Vegas, Macau, and Boston' : 'Las Vegas',
'Las Vegas, Macau, and Boston Harbor' : 'Las Vegas',
'Latin America and Asia Pacific' : 'Latin America and Asia Pacific',
'Latin America and Asia Pacific regions' : 'Latin America and Asia Pacific',
'Latin American markets and Western Europe' : 'Latin American markets and Western Europe',
'Macau, Las Vegas, Boston Harbor' : 'Las Vegas',
'Massachusetts' : 'Massachusetts',
'North America' :'North America' ,
'North America and Asia Pacific markets' : 'North America and Asia Pacific',
'North America and Asia Pacific region' : 'North America and Asia Pacific',
'North America and Australasia' : 'North America and Australasia',
'North America and Europe' : 'Europe and the United States',
'North America and International markets' : 'North America',
'North America and international markets' : 'North America',
'North America, Latin America' : 'North America, Latin America',
'North America, with offsetting trends in Europe and China' : 'North America, with offsetting trends in Europe and China',
'Primarily in the United States' : 'United States',
'U.S. and Europe' : 'Europe and the United States',
'United States' : 'United States',
'Western Europe' : 'Europe',
'Worldwide' : 'Global',
'Wynn Palace, Wynn Macau, and Las Vegas Operations' : 'Las Vegas'}

dic_when_risk = {}

dic_where_risk = {
'Offshore locations' : 'Offshore locations',
'Other countries' : 'Other countries' ,
'Third world countries' : 'Third world countries'}

dic_who_risk = {
'A conservative politician':'A conservative politician',
'Businesses':'Businesses',
'Clients':'Clients',
'Clients and potential clients':'Clients',
'Consumers':'Consumers',
'Consumers and investors':'Consumers',
'Consumers:, Data Providers':'Consumers',
'Data providers':'Data providers',
'Data providers and businesses':'Data providers',
'Domestic firms':'Domestic firms',
'Domestic unskilled workers':'Domestic unskilled workers',
'Economists':'Economists',
'End users:, main service providers:, IT providers': 'End users:, main service providers:, IT providers',
'Entrepreneurs':'Entrepreneurs',
'Foreign data providers':'Foreign firms',
'Foreign firms':'Foreign firms',
'Government':'Government',
'Government forces':'Government',
'IT industries':'IT industries',
'IT niche competitors':'IT industries',
'IT players':'IT industries',
'Individuals':'Individuals',
'Investors and consumers':'Consumers',
'Liberal politician or party':'Liberal politician or party',
'Local and multinational businesses':'Local and multinational businesses',
'Local businesses':'Local businesses',
'Local corporations':'Local businesses',
'Multinational companies':'Multinational companies',
'Multinational data providers':'Multinational companies',
'Multinational players':'Multinational companies',
'Net exporter countries':'Net exporter countries',
'Net importer countries':'Net importer countries',
'Political figureheads and parties':'Political figureheads and parties',
'Regulators':'Regulators',
'Researchers':'Researchers',
'Smaller domestic providers':'Smaller domestic providers',
'Startups and new businesses':'Startups and new businesses',
'Suppliers':'Suppliers',
'Talent':'Talent',
'Tectonic plates':'Tectonic plates',
'The government': 'Government'}

dic_where_twitter = {
' Helensvale Station': 'Helensvale Station',
'#gc2018': 'Gold Coast',
'#gc2018 (likely the Gold Coast 2018 Commonwealth Games)': 'Gold Coast',
'#india': 'India',
'37 nations': '37 nations',
'@theadelaideoval': 'Adelaide Oval',
'A coffee shop':'A coffee shop',
'A playground in the town of West Seneca:, New York': 'New York',
'A village': 'village',
'Adelaide':'Adelaide Oval',
'Adelaide:, Australia':'Adelaide Oval',
'Airport':'Airport',
'Auckland:, New Zealand': 'Auckland',
'Australia':'Australia',
'Australia-run detention centers':'Australia',
'Back alley karaoke bar':'Back alley karaoke bar',
'Bayazama':'Bayazama',
'Beaches (Gold Coast)': 'Gold Coast',
'Benleigh':'Benleigh',
'Between Yerongpilly and Kuraby stations':'Between Yerongpilly and Kuraby stations',
'Birmingham':'Birmingham',
'Birmingham:, England':'Birmingham',
'Borobudur (implied by hashtag)':'Borobudur',
'Brisbane':'Brisbane',
'Brisbane Airport':'Brisbane',
'Brisbane or Gold Coast': 'Gold Coast',
'Britain':'Britain',
'Broadbeach Bowls Club':'Broadbeach Bowls Club',
'Burleigh':'Burleigh',
'CG2018 (location not specified)': 'Gold Coast',
'Cairns or Townsville':'Cairns or Townsville',
'Canada':'Canada',
'Carara':'Carara',
'Carara Stadium':'Carara',
'Caribbean Sea:, aboard the Norwegian Epic cruise ship':'Caribbean Sea:, aboard the Norwegian Epic cruise ship',
'Comera station':'Comera station',
'Commonwealth Games': 'Gold Coast',
'Commonwealth Games 2018': 'Gold Coast',
'Commonwealth Games 2018 (#gc2018)': 'Gold Coast',
'Commonwealth Games 2018 Opening Ceremony':'Gold Coast',
'Commonwealth Games closing ceremony':'Gold Coast',
'Commonwealth Games in Australia':'Gold Coast',
'Commonwealth Games opening ceremony':'Gold Coast',
'Commonwealth Games swimming pool':'Gold Coast',
'Commonwealth Games venue (Specific location not mentioned)':'Gold Coast',
'Crowded events':'Crowded events',
'Delhi':'Delhi',
'Deprived areas':'Deprived areas',
'England':'England',
'Foreign country':'Foreign country',
"Fredie's Bar":"Fredie's Bar",
'Freshwater rivers and lakes':'Freshwater rivers and lakes',
'GC night clubs':'Gold Coast',
'GC2018 (Commonwealth Games 2018)':'Gold Coast',
'GC2018 (Gold Coast Commonwealth Games)':'Gold Coast',
'Games Village':'Gold Coast',
'Global':'Global',
'Gold Coast':'Gold Coast',
'Gold Coast (GC)':'Gold Coast',
'Gold Coast (implied by #gc2018)':'Gold Coast',
'Gold Coast (implied by #getingaroundgc)':'Gold Coast',
'Gold Coast 2018 Commonwealth Games':'Gold Coast',
'Gold Coast 2018 Commonwealth Games (#GC2018)':'Gold Coast',
'Gold Coast Line':'Gold Coast',
'Gold Coast and Benleigh line:, between Central and Varsity Lakes stations':'Gold Coast',
'Gold Coast to Carara':'Gold Coast',
'Gold Coast:, 2018 Commonwealth Games':'Gold Coast',
'Gold Coast:, Australia':'Gold Coast',
'Gold Coast:, Australia (implied by #gc2018)':'Gold Coast',
'Gold Coast:, Australia (implied)':'Gold Coast',
'Gold Coast:, Australia (specifically on the GC Hwy)':'Gold Coast',
'Gold Coast/Benleigh train line':'Gold Coast',
'Haryana:, India': 'India',
'Have-a-go zone':'Have-a-go zone',
'Helensvale station':'Helensvale station',
'Host city of the Commonwealth Games (not specified)':'Gold Coast',
'Humbleton':'Humbleton',
'IAF Junior Championship':'IAF Junior Championship',
'India':'India',
'Jamaica':'Jamaica',
'Jungle':'Jungle',
'Kashmir':'Kashmir',
'King George Square':'King George Square',
'Labrador and Southport areas':'Labrador and Southport areas',
'Logan City:, Queensland (implied)':'Queensland',
'Loganlea':'Loganlea',
'London': 'England',
'London:, England':'England',
'Los Angeles:, California': 'Los Angeles',
'Loughborough':'Loughborough',
'Makeup room':'Makeup room',
'Malawi':'Malawi',
'Melbourne:, Australia':'Australia',
'Montreal':'Montreal',
'Near the city of Lod in central Israel':'Near the city of Lod in central Israel',
'Nerang mountain bike trails':'Nerang mountain bike trails',
'New Zealand':'New Zealand',
'North of Caboolture':'North of Caboolture',
'Northern Gold Coast': 'Northern Gold Coast',
'Not mentioned:, but likely in India based on the hashtags': 'India', 
'Nyonyo Gardens':'Nyonyo Gardens',
'Office':'Office',
'Olympics':'Olympics',
'Ontario:, Canada': 'Canada',
'Outside the stadium':'Outside the stadium',
'Oxenford':'Oxenford',
'Oxenford Studios': 'Oxenford',
'Pal Mal':'Pal Mal',
'Parkwood':'Parkwood',
'Parts of India':'Parts of India',
'Perybar':'Perybar',
'Queensland': 'Queensland',
'Queensland:, Australia': 'Queensland',
'Quen St': 'Queensland',
'Redcar': 'Redcar',
'Redcliffe': 'Redcliffe',
'Rio Games': 'Rio Games',
'SEQ Rail network': 'SEQ Rail network',
'Samoa': 'Samoa',
'San Francisco': 'San Francisco',
'Sheffield': 'Sheffield',
'South Bank': 'South Bank',
'Southport': 'Southport',
'Southport Precinct:, Quenstre Village': 'Southport Precinct:, Quenstre Village',
"Speaker's workplace trainstop": "Speaker's workplace trainstop",
'Stadium': 'Stadium',
'Stadium for the Commonwealth Games (#comgames)': 'Gold Coast',
'Stretford': 'Stretford',
'Sunshine Coast': 'Sunshine Coast',
'Surfers Paradise:, Australia': 'Australia',
'Syria': 'Syria',
'Tehran': 'Tehran',
'The marshes': 'The marshes',
'The town the speaker lives in': 'The town the speaker lives in',
'The village': 'village',
'Tokyo': 'Tokyo',
'Tokyo:, Japan': 'Tokyo',
'Towards Australia Fair': 'Australia',
'Training venue for the Commonwealth Games': 'Training venue for the Commonwealth Games',
'Uganda': 'Uganda',
'Uni': 'Uni',
'United States': 'United States',
'Varsity Lakes': 'Varsity Lakes',
'Varsity Lakes Station': 'Varsity Lakes',
'Wakefield': 'Wakefield',
'Wales and England': 'England',
'Western Europe': 'Western Europe',
'Woolworths':'Woolworths',
'Yerogpily and Roma Stret': 'Yerogpily and Roma Stret',
'Yorkshire': 'Yorkshire'}

dic_when_twitter = {
'2006-03-15': ['2006'],
'2016':['2016'],
'2017':['2017'],
'2018':['2018'],
'2018-04-04':['2018'],
'2018-04-07':['2018'],
'2019':['2019'],
'2020':['2020'],
'2021-07-23':['2021'],
'2021-07-23 to 2021-08-08':['2021'],
'2022':['2022'],
'2022-03-27':['2022'],
'2022-07-28 to 2022-08-08':['2022'],
'2023-04-23 19:00':['2023'],
'2023-06-14':['2023'],
'2023-10-27 between 6-9 PM':['2023'],
'After Gold Coast 2018':['After Gold Coast 2018'],
'April 5-15': ['2015'],
'Before the games':['After Gold Coast 2018'],
'During #glasgow2014':['During Gold Coast 2018'],
'June':['June']}

dic_when_headlines = {
'1110':['1110'],
'1110 AD':['1110'],
'1916-05-03':['1916'],
'1992':['1992'],
'2010':['2010'],
'2020':['2020'],
'2020-02-11':['2020'],
'2020-03-05':['2020'],
'2020-03-24 20:00':['2020'],
'2020-04-28':['2020'],
'2020-04-30':['2020'],
'2020-05-04':['2020'],
'2020-05-05':['2020'],
'2020-05-07':['2020'],
'2020-05-08':['2020'],
'2020-05-10':['2020'],
'2020-05-11':['2020'],
'2020-05-12':['2020'],
'2020-05-14':['2020'],
'2020-05-15':['2020'],
'2020-05-16':['2020'],
'2020-05-18':['2020'],
'2020-05-22':['2020'],
'2020-05-25':['2020'],
'2020-06-06':['2020'],
'2020-09-04':['2020'],
'2020-11-08 at 11:00 AM':['2020'],
'2021':['2021'],
'2021-05-19':['2021'],
'2023-04-17':['2023'],
'2023-04-29':['2023'],
'2023-05-01':['2023'],
'2023-05-08':['2023'],
'2023-05-11':['2023'],
'2023-05-12':['2023'],
'2023-05-13':['2023'],
'2023-05-14':['2023'],
'2023-05-15':['2023'],
'2023-05-18':['2023'],
'2023-05-19':['2023'],
'2023-05-20':['2023'],
'2023-05-22':['2023'],
'2023-05-30':['2023'],
'2023-06-01 to 2023-06-12':['2023'],
'2023-06-22':['2023'],
'2023-06-27':['2023'],
'2023-07-01':['2023'],
'2023-07-05 at 11:00 am':['2023'],
'2023-07-14':['2023'],
'2023-07-15':['2023'],
'2023-10-26':['2023'],
'2023-10-26 11:00 AM ET':['2023'],
'2023-10-26 20:00':['2023'],
'2023-10-26 4 PM':['2023'],
'2024':['2024'],
'After the 2020 season':['After the 2020 season'],
'April':['April'],
'April (implied)':['April'],
'April (specific date not mentioned)':['April'],
'April 2020':['April', '2020'],
'August (specific date not mentioned)':['August'],
'Could be done by June': ['June'],
'Every Monday at 9:00 AM':['Every Monday at 9:00 AM'],
'Every Saturday at 9:00 AM':['Every Saturday at 9:00 AM'],
'Expected to be operational by 2023': ['2023'],
'First quarter of 2020': ['2020'],
'First week of June': ['June'], 
'From May 12': ['May'],
'Holiday 2020': ['2020'],
'In or around June': ['June'],
'July (estimated)':['July'],
'July (specific date not mentioned)':['July'],
'July 2020': ['July', '2020'],
'June (approximate)': ['June'],
'June (estimated)':['June'],
'June (specific date not mentioned)':['June'],
'June 12':['June'],
'May (specific date not mentioned)':['May'],
'May 1st:, 2020': ['May','2020'],
'May 2020': ['May','2020'],
'May 2020 (week one)':['May','2020'],
'May 2020 week two':['May','2020'],
'May 2021': ['May','2021'],
'May 22': ['May'],
'May 23rd': ['May'],
'May 30': ['May'],
'May 8th': ['May'],
'Mid May': ['May'],
'Mid-1990s': ['1990'],
'Mid-May': ['May'],
'Q1 2020': ['2020'],
'September': ['September'],
'Since the start of lockdown':['Since the start of lockdown'],
'Summer 2020': ['2020'],
'Summer 2021':['2021'],
'Summer 2023':['2023'],
'This July': ['July'],
'Through June 30': ['June'],
'Through May': ['May'],
'Through May 2021': ['May', '2021'],
'Till May 10':['May'],
'Till May 29':['May'],
'Till May 31':['May'],
'Until 2020-05-21': ['2020'],
'Until 2023-05-05': ['2023'],
'Until 2024': ['2024'],
'Until July 31': ['July'],
'Until July 31:, 2020': ['July', '2020'],
'Until May 24': ['May'],
'Week to May 1': ['May']}


In [68]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine


def generate_dict(csv, ws):
    
    df = pd.read_csv(csv)
    l_w = []
    for l in df['5w1h_cause'].to_list():
        try:
            l_w.append(eval(l)[ws])
        except:
            l_w.append('Not mentioned')
    
    for l in df['5w1h_effect'].to_list():
        try:
            l_w.append(eval(l)[ws])
        except:
            l_w.append('Not mentioned')
    
    set_w = set(l_w)
    set_w.remove('Not mentioned')

    l_w = list(set_w)

    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    sentences = model.encode(l_w)

    dic_w = {}

    for i in range(len(sentences)):
        for j in range(i+1,len(sentences)):
            if cosine(sentences[i],sentences[j]) < 0.2:
                if l_w[j] not in dic_w:
                    dic_w[l_w[j]] = l_w[i]
    
    for key in l_w:
        if key not in dic_w:
            dic_w[key] = key

    return dic_w

In [69]:
dic_who_headlines = generate_dict('Headlines.csv', 'Who')
dic_where_headlines = generate_dict('Headlines.csv', 'Where')
dic_who_Twitter = generate_dict('Twitter.csv', 'Who')

In [61]:
import numpy as np
import networkx as nx
import pandas as pd

def generate_homogeneous_graph(df, model, cause_col = ' Cause', effect_col = ' Effect'):
   
   graph = nx.DiGraph()
   df[cause_col] = df[cause_col].astype(str)
   df[effect_col] = df[effect_col].astype(str)

   df['Embedding_' + cause_col] = list(model.encode(df[cause_col]))
   df['Embedding_' + effect_col] = list(model.encode(df[effect_col]))

   for _, row in df.iterrows():
      cause = row[cause_col]
      effect = row[effect_col]
      
      graph.add_edge('event:' + cause, 'relation: ' + cause + '_' + effect)
      graph.add_edge('relation: ' + cause + '_' + effect, 'event:' + effect)
      
      graph.nodes['event:' + cause]['label'] = 'aux'
      graph.nodes['event:' + effect]['label'] = 'aux'
      graph.nodes['relation: ' + cause + '_' + effect]['label'] = row['Label']

      graph.nodes['event:' + cause]['embedding'] = np.asarray(row['Embedding_' + cause_col], dtype=np.float64)
      graph.nodes['event:' + effect]['embedding'] = np.asarray(row['Embedding_' + effect_col], dtype=np.float64)
      graph.nodes['relation: ' + cause + '_' + effect]['embedding'] = np.mean([np.asarray(row['Embedding_' + cause_col], dtype=np.float64),np.asarray(row['Embedding_' + effect_col], dtype=np.float64)], axis=0)

   return graph
              
def generate_heterogeneous_graph(df, model, dic_who, dic_when, dic_where, cause_col = ' Cause', effect_col = ' Effect'):

   graph = nx.Graph()

   df[cause_col] = df[cause_col].astype(str)
   df[effect_col] = df[effect_col].astype(str)

   df['Embedding_' + cause_col] = list(model.encode(df[cause_col]))
   df['Embedding_' + effect_col] = list(model.encode(df[effect_col]))

   for _, row in df.iterrows(): 
      
      cause = row[cause_col]
      effect = row[effect_col]
      
      graph.add_edge('event:' + cause, 'relation: ' + cause + '_' + effect)
      graph.add_edge('relation: ' + cause + '_' + effect, 'event:' + effect)
      
      graph.nodes['event:' + cause]['label'] = 0#'aux'
      graph.nodes['event:' + effect]['label'] = 0#'aux'
      graph.nodes['relation: ' + cause + '_' + effect]['label'] = 1#row['Label']

      graph.nodes['event:' + cause]['embedding'] = np.asarray(row['Embedding_' + cause_col], dtype=np.float64)
      graph.nodes['event:' + effect]['embedding'] = np.asarray(row['Embedding_' + effect_col], dtype=np.float64)
      graph.nodes['relation: ' + cause + '_' + effect]['embedding'] = np.mean([np.asarray(row['Embedding_' + cause_col], dtype=np.float64),np.asarray(row['Embedding_' + effect_col], dtype=np.float64)], axis=0)
      
      topic_cause = row['Topic_Cause']
      topic_effect = row['Topic_Effect']
      try:
         who_cause = eval(row['5w1h_cause'])['Who']
         where_cause = eval(row['5w1h_cause'])['Where']
         when_cause = eval(row['5w1h_cause'])['When']
      except:
         who_cause = ''
         where_cause = ''
         when_cause = ''
      try:
         who_effect = eval(row['5w1h_effect'])['Who']
         where_effect = eval(row['5w1h_effect'])['Where']
         when_effect = eval(row['5w1h_effect'])['When']
      except:
         who_effect = ''
         where_effect = ''
         when_effect = ''

      if topic_cause != '-1_the_in_and_of': 
         graph.add_edge('event:' + cause, 'topic:' + topic_cause)
         graph.add_edge('topic:' + topic_cause, 'event:' + cause)
         graph.nodes['topic:' + topic_cause]['embedding'] = np.asarray(model.encode(topic_cause), dtype=np.float64)
         graph.nodes['topic:' + topic_cause]['label'] = 2#'topic'

      if topic_effect != '-1_the_in_and_of': 
         graph.add_edge('event:' + effect, 'topic:' + topic_effect)
         graph.add_edge('topic:' + topic_effect,'event:' + effect)
         graph.nodes['topic:' + topic_effect]['embedding'] = np.asarray(model.encode(topic_effect), dtype=np.float64)
         graph.nodes['topic:' + topic_effect]['label'] = 2#'topic'
         
      try:
         graph.add_edge('event:' + cause, 'who:' + dic_who[who_cause])
         graph.add_edge('who:' + dic_who[who_cause],'event:' + cause)
         graph.nodes['who:' + dic_who[who_cause]]['embedding'] = np.asarray(model.encode(dic_who[who_cause]), dtype=np.float64)
         graph.nodes['who:' + dic_who[who_cause]]['label'] = 3#'who'
      except: 
         print('Dict who without the key: ' + who_cause)

      try:
         graph.add_edge('event:' + cause, 'where:' + dic_where[where_cause])
         graph.add_edge('where:' + dic_where[where_cause], 'event:' + cause)
         graph.nodes['where:' + dic_where[where_cause]]['embedding'] = np.asarray(model.encode(dic_where[where_cause]), dtype=np.float64)
         graph.nodes['where:' + dic_where[where_cause]]['label'] = 4#'where'
      except: 
         print('Dict where without the key: ' + where_cause)

      try:
         wcs = dic_when[when_cause]
         for wc in wcs:
            graph.add_edge('event:' + cause, 'when:' + wc)
            graph.add_edge('when:' + wc, 'event:' + cause)
            graph.nodes['when:' + wc]['embedding'] = np.asarray(model.encode(wc), dtype=np.float64)
            graph.nodes['when:' + wc]['label'] = 5#'when'
      except: 
         print('Dict wheen without the key: ' + when_cause)

      try:
         graph.add_edge('event:' + effect, 'who:' + dic_who[who_effect])
         graph.add_edge('who:' + dic_who[who_effect],'event:' + effect)
         graph.nodes['who:' + dic_who[who_effect]]['embedding'] = np.asarray(model.encode(dic_who[who_effect]), dtype=np.float64)
         graph.nodes['who:' + dic_who[who_effect]]['label'] = 3#'who'
      except: 
         print('Dict who without the key: ' + who_effect)

      try:
         graph.add_edge('event:' + effect, 'where:' + dic_where[where_effect])
         graph.add_edge('where:' + dic_where[where_effect],'event:' + effect)
         graph.nodes['where:' + dic_where[where_effect]]['embedding'] = np.asarray(model.encode(dic_where[where_effect]), dtype=np.float64)
         graph.nodes['where:' + dic_where[where_effect]]['label'] = 4#'where'
      except: 
         print('Dict where without the key: ' + where_effect)

      try:
         wes = dic_when[when_effect]
         for we in wes:
            graph.add_edge('event:' + effect, 'when:' + we)
            graph.add_edge('when:' + we,'event:' + effect)
            graph.nodes['when:' + we]['embedding'] = np.asarray(model.encode(we), dtype=np.float64)
            graph.nodes['when:' + we]['label'] = 5#'when'
      except: 
         print('Dict wheen without the key: ' + when_effect)
   
   return graph

In [43]:
from sentence_transformers import SentenceTransformer
import pickle as pkl

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

df_fincausal = pd.read_csv('FinCausal.csv')

df_risk = pd.read_csv('Risk.csv')

df_twitter = pd.read_csv('Twitter.csv')

df_headlines = pd.read_csv('Headlines.csv')

In [ ]:
graph_fincausal = generate_heterogeneous_graph(df_fincausal, model, dic_who_fincausal, dic_when_fincausal, dic_where_fincausal)

graph_risk = generate_heterogeneous_graph(df_risk, model, dic_who_risk, dic_when_risk, dic_where_risk)

graph_twitter = generate_heterogeneous_graph(df_twitter, model, dic_who_twitter, dic_when_twitter, dic_where_twitter)

graph_headlines = pd.read_csv('Headlines.csv')

In [58]:
from plotly import graph_objs as go

def show_graph(G):
  ### ARESTAS
  edge_x = []
  edge_y = []

  # adicionando as coordenadas
  for edge in G.edges():
      x0, y0 = G.nodes[edge[0]]['pos']
      x1, y1 = G.nodes[edge[1]]['pos']
      edge_x.append(x0)
      edge_x.append(x1)
      edge_x.append(None)
      edge_y.append(y0)
      edge_y.append(y1)
      edge_y.append(None)

  # definindo cor e estilo das arestas
  edge_trace = go.Scatter(
      x=edge_x, y=edge_y,
      line=dict(width=2, color='#888'),
      hoverinfo='none',
      mode='lines')

  ### VÉRTICES
  node_x = []
  node_y = []

  # adicionando as coordenadas
  #text = []
  for node in G.nodes():
      x, y = G.nodes[node]['pos']
      node_x.append(x)
      node_y.append(y)
      #text.append(G.nodes[node]['text']) 

  # definindo cor e estilo dos vértices
  node_trace = go.Scatter(
      x=node_x, y=node_y,
      mode='markers',
      hoverinfo='text',
      #hovertext= text,
      marker=dict(
          size=10,
          line_width=2))
  
  node_labels = []
  for node in G.nodes():
    node_labels.append(G.nodes[node]['label'])

  node_trace.marker.color = node_labels
  
  # visualizando!
  fig = go.Figure(data=[edge_trace, node_trace],
              layout=go.Layout(
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20,l=5,r=5,t=40),
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  fig.show()

In [63]:
pos = nx.spring_layout(graph_twitter,seed=81) # obtém coordenadas dos vértices para visualização
for node in graph_twitter.nodes():
  graph_twitter.nodes[node]['pos'] = pos[node]

In [64]:
show_graph(graph_twitter)

In [65]:
pos = nx.spring_layout(graph_risk,seed=81) # obtém coordenadas dos vértices para visualização
for node in graph_risk.nodes():
  graph_risk.nodes[node]['pos'] = pos[node]

show_graph(graph_risk)

In [67]:
pos = nx.spring_layout(graph_fincausal,seed=81) # obtém coordenadas dos vértices para visualização
for node in graph_fincausal.nodes():
  graph_fincausal.nodes[node]['pos'] = pos[node]

show_graph(graph_fincausal)